<a href="https://colab.research.google.com/github/anmolag190153/BCS_summer_project_SER/blob/main/Speech_emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SPEECH EMOTION RECOGNITION PROJECT
## BCS-IITK


# Emotions Classification using MLP model

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.callbacks import CSVLogger
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# path1='/content/drive/My Drive/Ravdess/Actor_01/03-01-08-02-02-02-01.wav'
# ipd.Audio(path1)

In [6]:
# actor_1,sr=librosa.load(path1)
# print(actor_1.shape)
# print(sr)
sr=22100

In [7]:
# plt.figure(figsize=(15,5))
# librosa.display.waveplot(actor_1,sr=sr)
# plt.ylim((-0.1,0.1))
# plt.show()

In [8]:
def build_fc_model(input_shape=(180,1), num_class=8):
    
    
    model=tf.keras.models.Sequential([tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(512,activation='relu',input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(1e-2)),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(512,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(1e-3)),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(128,activation='relu'),
                                    tf.keras.layers.Dense(64,activation='relu'),
                                    tf.keras.layers.Dense(num_class,activation='softmax')
                                    ])

    return model

In [9]:
def extract_feature(file):
    result=np.array([])
    mfcc=np.mean(librosa.feature.mfcc(file,sr=sr,n_mfcc=40).T,axis=0)
    result=np.hstack((result,mfcc))
    chroma=np.mean(librosa.feature.chroma_stft(file,sr=sr).T,axis=0)
    result=np.hstack((result,chroma))
    mel=np.mean(librosa.feature.melspectrogram(file,sr=sr).T,axis=0)
    result=np.hstack((result,mel))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(file, sr=sr).T,axis=0)
    result=np.hstack((result,spectral_rolloff))
    return result

# Loading Data
We are using the RAVDESS dataset for this project.

In [14]:
path='Audio_Datasets'
k=0
x_data=np.ndarray(shape=(24*60,120000))
y_data=np.ndarray(shape=(24*60,1))
for i,actor in enumerate(os.listdir(path)):
    path1=os.path.join(path,actor)
    for j,file in enumerate(os.listdir(path1)):
        b=file[6:8]
        y_data[k,0]=b
        y_data[k,0]=y_data[k,0]-1
        temp_path=os.path.join(path1,file)
        # print(temp_path)
        var,_=librosa.load(temp_path)
        x_data[k,:var.shape[0]]=var[:]
        x_data[k,var.shape[0]:]=0
        k=k+1

# Extracting Features
We have used mfccs, chroma frequency, mel spectogram and spectral centroid as audio features in this project.

In [8]:
x=[]
for i in range(24*60):
    feature=extract_feature(x_data[i,:])
    x.append(feature)

**Scaling** the data and splitting data for **training** and **validation**

In [9]:
x_train, x_test,y_train,y_test = train_test_split(np.array(x), y_data, test_size=0.2, random_state=9)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Training the model
The model classifies human voices in 8 different classes of emotions

**label**   **emotion**

0   =   neutral

1   =   calm

2   =   happy

3   =   sad

4   =   angry

5   =   fearful

6   =   disgust

7   =   surprised

In [34]:
model=build_fc_model()
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
csv_logger = CSVLogger('model_history.log', separator=',', append=False)
history=model.fit(x_train,y_train,epochs=500,batch_size=256,validation_data=(x_test,y_test),callbacks=[csv_logger])

===========================] - 0s 12ms/step - loss: 0.3396 - accuracy: 0.9575 - val_loss: 1.1300 - val_accuracy: 0.7500
Epoch 361/500
5/5 [==============================] - 0s 12ms/step - loss: 0.3305 - accuracy: 0.9609 - val_loss: 1.1920 - val_accuracy: 0.7535
Epoch 362/500
5/5 [==============================] - 0s 11ms/step - loss: 0.3053 - accuracy: 0.9696 - val_loss: 1.2309 - val_accuracy: 0.7326
Epoch 363/500
5/5 [==============================] - 0s 11ms/step - loss: 0.3173 - accuracy: 0.9653 - val_loss: 1.2083 - val_accuracy: 0.7049
Epoch 364/500
5/5 [==============================] - 0s 12ms/step - loss: 0.3063 - accuracy: 0.9722 - val_loss: 1.0858 - val_accuracy: 0.7569
Epoch 365/500
5/5 [==============================] - 0s 12ms/step - loss: 0.3012 - accuracy: 0.9670 - val_loss: 1.1726 - val_accuracy: 0.7465
Epoch 366/500
5/5 [==============================] - 0s 14ms/step - loss: 0.2937 - accuracy: 0.9731 - val_loss: 1.1349 - val_accuracy: 0.7431
Epoch 367/500
5/5 [=========

In [31]:
model.save('mlp_model_Team-4')

INFO:tensorflow:Assets written to: mlp_model_Team-4/assets


In [36]:
result = model.evaluate(x_test,y_test)
print('Accuracy obtained on validation is ',result[1]*100,'%')

9/9 [==============================] - 0s 2ms/step - loss: 1.3521 - accuracy: 0.7118
Accuracy obtained on validation is  71.18055820465088 %
